In [18]:
# prerequisite: 
# run 
#      ./printEnergyCorrections -s ../../setup/setup.json -t ../../setup/target.json -d U4,U5 -E {insert peaks one wishes to fit after} -i {implantation depth of given isotope} > {filename}.dat
# with data output to
#      "correctedEnergiesU4U5.dat"

In [2]:
import numpy as np
import uproot
import matplotlib.pyplot as plt

In [17]:
#load in generated file from above
Es = np.loadtxt('/home/haue/repositories/Masters/calibration/dssd_calibration/U5_sipeaks_corr.dat').flatten()
print(len(Es)/4)

49.0


In [4]:
from scipy.optimize import curve_fit
def lin(x, a, b):
    return a*x + b

In [ ]:
"""
([(385.72237105453763, 0.5786771607068808),
  (1843.1815428519299, 3.8391338032060935),
  (2217.4549852704345, 1.9202145004587217),
  (4651.192036484095, 4.798722937645029),
  (5402.617947801247, 1.9202145004587354)],
 [(904.0191131421628, 1.920220495723927),
  (2076.7425660534773, 3.8391368018535106),
  (3337.756164355824, 4.798725336660437),
  (4089.1820756729758, 1.920220495723936)],
 (1730.8841553184843, 3.8391483166378255),
"""
nums =  ["5"]
sides = ["F", "B"]
Es0 = np.array([904.02,1843.18, 2076.74, 2217.45, 3337.75,4089.18,5402.61])
Es0_errs = np.array([1.92,3.84, 3.84, 1.92, 4.79, 1.92, 1.92])

j = 0
for num in nums:
        for side in sides:
            for strip in range(2, 15+1, 1):
                
                print([Es[7*j+i] for i in range(len(Es0))])
                j += 1

[877.588, 1827.59, 2062.39, 2203.7, 3327.59, 4080.63, 5395.59]
[878.145, 1827.88, 2062.65, 2203.95, 3327.77, 4080.78, 5395.71]
[880.842, 1829.3, 2063.9, 2205.14, 3328.64, 4081.5, 5396.3]
[871.35, 1823.84, 2058.91, 2200.37, 3325.13, 4078.56, 5393.89]
[867.4, 1821.5, 2056.75, 2198.3, 3323.61, 4077.28, 5392.84]
[864.682, 1819.88, 2055.25, 2196.86, 3322.55, 4076.39, 5392.11]
[862.507, 1818.57, 2054.05, 2195.7, 3321.69, 4075.67, 5391.53]
[860.68, 1817.48, 2053.04, 2194.74, 3320.98, 4075.07, 5391.03]
[859.141, 1816.57, 2052.19, 2193.92, 3320.38, 4074.57, 5390.62]
[857.864, 1815.81, 2051.49, 2193.25, 3319.88, 4074.15, 5390.28]
[856.826, 1815.19, 2050.92, 2192.71, 3319.48, 4073.81, 5390.0]
[855.997, 1814.71, 2050.47, 2192.28, 3319.17, 4073.55, 5389.78]
[855.357, 1814.33, 2050.13, 2191.95, 3318.93, 4073.35, 5389.62]
[854.929, 1814.09, 2049.92, 2191.75, 3318.77, 4073.22, 5389.51]
[862.695, 1818.71, 2054.18, 2195.83, 3321.78, 4075.75, 5391.58]
[859.568, 1816.85, 2052.47, 2194.19, 3320.58, 4074.73

In [19]:
"""
dead_strips = ["U5F2"]
def count_in_range(xs, low, high):
    return np.sum((xs >= low) & (xs <= high))
nums =  ["5"]
sides = ["F", "B"]
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0)
                    j += 1
                    continue
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                print("x3 and y3 values", list(zip(x3,y3)))

                while len(x3_sortedx) > len(Es0) + 1:
                    # Count how many peaks still remain in protected regions
                    count_230_280 = count_in_range(x3_sortedx, 230, 280)
                    count_500_650 = count_in_range(x3_sortedx, 500, 650)

                    # Build mask of deletable peaks (i.e., not in protected zones)
                    deletable = np.ones(len(x3_sortedx), dtype=bool)
                    for i, xval in enumerate(x3_sortedx):
                        if 230 <= xval <= 280 and count_230_280 <= 1:
                            deletable[i] = False
                        if 500 <= xval <= 650 and count_500_650 <= 2:
                            deletable[i] = False

                    if not np.any(deletable):
                        break  # nothing deletable left, exit

                    # Choose lowest y among deletable peaks
                    masked_y = np.where(deletable, y3_sortedx, np.inf)
                    idx = np.argmin(masked_y)

                    # Update counters if removing a protected peak
                    if 230 <= x3_sortedx[idx] <= 280:
                        count_230_280 -= 1
                    if 500 <= x3_sortedx[idx] <= 650:
                        count_500_650 -= 1

                    # Remove peak
                    x3_sortedx = np.delete(x3_sortedx, idx)
                    y3_sortedx = np.delete(y3_sortedx, idx)
                    
                    
                plt.figure()
                plt.plot(x2,y2)
                for x,y in zip(x3_sortedx,y3_sortedx):
                    plt.plot(x, y, 'rs')
                plt.xlim(0, 2000)
                plt.title(base)
                plt.show()
                plt.close()
"""

'\ndead_strips = ["U5F2"]\ndef count_in_range(xs, low, high):\n    return np.sum((xs >= low) & (xs <= high))\nnums =  ["5"]\nsides = ["F", "B"]\nwith uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:\n    for num in nums:\n        for side in sides:\n            if side == "F":\n                print("# Front")\n            else:\n                print("# Back")\n            for strip in range(1, 16+1, 1):\n                if strip == 1 or strip == 16:\n                    print(0, 0)\n                    continue\n                base = "U" + num + side + str(strip)\n                #skip dead strips\n                if base in dead_strips:\n                    print(0, 0)\n                    j += 1\n                    continue\n                y1, x1 = file[base].to_numpy()\n                x0 = np.min(x1) ## starting bin, in this case just 90\n                x1 = x1[:-1] + 0.5\n                # tspectrum\n                y2, x2 = file[base + "A"].to_numpy

In [27]:
dead_strips = ["U5F2"]
def count_in_range(xs, low, high):
    return np.sum((xs >= low) & (xs <= high))
nums =  ["5"]
sides = ["F", "B"]
j = 0
res0 = res = np.array([])
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0)
                    j += 1
                    continue
                # data
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                # fitted peak positions and their errors
                fx, fx_err = file[base + "F"].values()

                
                # peak positions, sorted xmin to xmax
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                fx = fx[np.argsort(fx)]
                fx_err = fx_err[np.argsort(fx)]
                
                if len(x3_sortedx)!=len(fx):
                    raise ValueError(f"Length mismatch: len(x3_sortedx)={len(x3_sortedx)} is not equal to len(fx)={len(fx)}, somehow the fitting procedure messed up..")



                # proton peaks pV, pVI, ..., pVIII
                ##Hand held peak pruning
                while len(x3_sortedx) > len(Es0):
                    # Count how many peaks still remain in protected regions
                    count_230_280 = count_in_range(x3_sortedx, 230, 280)
                    count_500_650 = count_in_range(x3_sortedx, 500, 650)

                    deletable = np.ones(len(x3_sortedx), dtype=bool)
                    for i, xval in enumerate(x3_sortedx):
                        if 230 <= xval <= 280 and count_230_280 <= 1:
                            deletable[i] = False
                        if 500 <= xval <= 650 and count_500_650 <= 2:
                            deletable[i] = False

                    if not np.any(deletable):
                        break  # nothing deletable left
                    
                    masked_y = np.where(deletable, y3_sortedx, np.inf)
                    idx = np.argmin(masked_y)

                    # Update counters if removing a protected peak
                    if 230 <= x3_sortedx[idx] <= 280:
                        count_230_280 -= 1
                    if 500 <= x3_sortedx[idx] <= 650:
                        count_500_650 -= 1
                        
                    x3_sortedx = np.delete(x3_sortedx, idx)
                    y3_sortedx = np.delete(y3_sortedx, idx)
                

                
                ### the gaussian fitting routine is very simple and therefore one should expect errors
                ### This would explain why x3_sortedx is the better parameter to use for now
                #chs = fx
                chs = x3_sortedx
                popt0, pcov0 = curve_fit(lin, chs, Es0, sigma=Es0_errs, p0=[2, 0])
                Ecs = np.array([Es[7*j+i] for i in range(len(x3_sortedx))])
                popt, pcov = curve_fit(lin, chs, Ecs, sigma=Es0_errs, p0=[2, 0])
                
                res0 = np.append(res0, Es0 - lin(chs, *popt0)) ## not energy corrected peak energies
                res  = np.append(res, Ecs - lin(chs, *popt)) ## energy corrected peak energies ! the most important one!
                
                
                ########################################################################################################################
                ## Plotting routine for chosen peaks
                """
                plt.figure()
                plt.plot(x2,y2)
                for x,y in zip(x3_sortedx,y3_sortedx):
                    plt.plot(x, y, 'rs')
                plt.xlim(0, 2000)
                plt.title(base)
                plt.show()
                plt.close()
                """              
                ######################################################################################################################
                ## plotting routine for quality control
                """
                fig, axes = plt.subplots(2, 1,gridspec_kw={'height_ratios': [4, 1], 'hspace': 0}, sharex=True)
                ax1, ax2 = axes
                #plt.sca(axes[0]) ## sets current axes
                ax1.errorbar(chs, Ecs, yerr=Es0_errs, capsize=2, marker='o',markersize=3.5, ls='', color='r', label="Data points")
                ax1.plot([50, 2000], lin(np.array([50, 2000]), *popt), label=f"fit vals: {np.round(popt[1],6)} {np.round(popt[0],6)}", color='k', ls='--')
                ax1.legend()
                ax1.set_ylabel("Energy(keV)")
                ax1.set_title(f"Linear fit to {base}")
                #plt.sca(axes[1])
                ax2.scatter(chs, Ecs - lin(chs, *popt), color='k', s=8)
                ax2.hlines(0,50,2000, ls='--', color='k', lw=0.7)
                ax2.set_yticks(np.arange(-10,10, 2.5), minor=True)
                ax2.set_ylim(-np.max(np.abs(Ecs - lin(chs, *popt)))-2, np.max(np.abs(Ecs - lin(chs, *popt)))+2)
                
                ax2.set_ylabel("Residuals (keV)")
                ax2.set_xlabel("Channel")
                
                plt.show()
                
                
                """
                #### output
                print(popt[1], popt[0])
                j += 1


# Front
0 0
0 0
-33.993881772204254 3.9000925505862027
-35.47973223162972 3.729869239960938
-44.923426827235794 3.6444485749911326
-56.02569007185556 3.67874173690302
-40.30862739181803 3.683154136436471
-56.21266220682167 3.629615300067444
-49.236307382037744 3.7034090552506833
-32.7265875424067 4.011168548154446
-47.21911777199593 3.6421479949841404
-29.422483281056714 3.671162824750678
-43.15175942076015 3.6727211991782163
-39.85316917906298 3.971160322420217
-52.55321649879875 3.8074978470749787
0 0
# Back
0 0
-49.194005152353846 3.7696536714300337
-26.435975758107638 3.682678988766708
-33.92568868168106 3.7672413624612124
-20.84612843571035 3.807064748385272
-44.311765430636406 3.683010720728484
-23.11989591937422 3.613250869396276
-38.21598600930362 4.072685330888521
-64.22674395589546 3.8221664128324413
-45.11620767780783 3.9015829690565353
-64.50358142471572 3.9405385412540817
-49.064201106191895 3.75568082926308
-61.247112824688756 3.729280708214658
-43.31644419285293 3.797034

In [ ]:
np.savetxt("res_proton_uncorrected.dat", res0)
np.savetxt("res_proton_corrected.dat", res)